In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-2/flax/gemma2-2b-it/1/tokenizer.model
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/_METADATA
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/manifest.ocdbt
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/checkpoint
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/_CHECKPOINT_METADATA
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/ocdbt.process_0/manifest.ocdbt
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/ocdbt.process_0/d/834bb4bf1e3854eb09f6208c95c071b2
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/ocdbt.process_0/d/bf69258061ae5f35eb7a5669fe6877d4
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/ocdbt.process_0/d/fc20151969d7ca91ea9d8275bda0e219
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/descriptor/descriptor.pbtxt
/kaggle/input/gemma-2/flax/gemma2-2b-it/1/gemma2-2b-it/d/b5a4695f4be0a2f41ec1e25616ebd7e7
/kaggle/input/google-tunix-hackathon/Hackathon dataset.txt
/kaggle/input/gemma/flax/2b-i

In [2]:
# Suppress asyncio error messages (they are non-critical)
import logging
logging.getLogger('asyncio').setLevel(logging.CRITICAL)
print("✓ Asyncio logging suppressed")

✓ Asyncio logging suppressed


## Tunix (Google’s new JAX-native library)

"Teach a LLM to reason using Tunix, Google’s new JAX-native library for LLM post-training."

"Most open-source or open-weight language models can give you an answer. But they typically **don’t ‘show their work’** - the steps they went through to arrive at that conclusion in a consistent manner. Here, you’ll use Tunix, Google’s new JAX-native library for LLM post-training, to train a model to show its work by laying out a reasoning trace before landing on an answer."

## Competition Citation

@misc{google-tunix-hackathon,
    author = {Wei Wei and María Cruz},
    title = {Google Tunix Hack - Train a model to show its work},
    
    year = {2025},
    howpublished = {\url{https://kaggle.com/competitions/google-tunix-hackathon}},
    note = {Kaggle}
}

## Tunix brought "Don't show their work" to a stratospheric level.

No outputs to the text file :) 

In [3]:
# http://www.gutenberg.org/cache/epub/730/pg730.txt
tunix = open("../input/google-tunix-hackathon/Hackathon dataset.txt", "r", encoding ='cp1251').read()
print (tunix[:1000])

## Tunix on TPU

"In this hackathon, you’ll start with Google’s open-weight Gemma model (Gemma2 2B or Gemma3 1B), fine-tune it with **Tunix on TPU**, and teach it how to reason through **complex questions**. You’ll create a model that not only gets the right answer, but also explains **how it got there**."

"The solution should include a working training pipeline using **Tunix and Gemma**. Share your configs, reward function composition, and recipes so others can reproduce your results and build on them."

https://www.kaggle.com/competitions/google-tunix-hackathon/overview

## Install Libraries

In [5]:
# Install packages in correct order to avoid conflicts
!pip install -q wandb
!pip install -q kagglehub
!pip install -q ipywidgets
!pip install -q tensorflow
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q transformers
!pip install -q grain
!pip install -q datasets

# Uninstall existing flax/jax
!pip uninstall -q -y flax jax jaxlib

# Install google-tunix which will pull correct versions
!pip install -q "google-tunix[prod]==0.1.3"

# Verify versions
import jax
import flax
print(f"JAX version: {jax.__version__}")
print(f"Flax version: {flax.__version__}")


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip



## Attach Secret

In [6]:
import wandb, os

try:
    from kaggle_secrets import UserSecretsClient
    os.environ['WANDB_API_KEY'] = UserSecretsClient().get_secret("WANDB_API_KEY")
    print("✓ WandB API key loaded")
except:
    print("⚠ WandB API key not available (skip if submission)")
    pass

✓ WandB API key loaded


## Import humanize

In [8]:
from flax.nnx import Variable

# Qwix/Flax 0.12.0 互換性パッチ
_original_set_metadata = Variable.set_metadata

def _patched_set_metadata(self, *args, **kwargs):
    """
    Qwix uses old Flax API: variable.set_metadata('key', value)
    Flax 0.12.0 uses new API: variable.set_metadata(key=value)
    This patch converts old format to new format.
    """
    if len(args) == 2 and len(kwargs) == 0:
        # Old API format detected - convert to new format
        key, value = args
        kwargs = {key: value}
        args = ()
    return _original_set_metadata(self, *args, **kwargs)

Variable.set_metadata = _patched_set_metadata
print("✓ Qwix/Flax 0.12.0 compatibility patch applied")

✓ Qwix/Flax 0.12.0 compatibility patch applied


In [7]:
import functools
import gc
import os
from pprint import pprint
import re
import csv
import shutil

# Import flax first, then configure
import flax
from flax import nnx

# REMOVE this line - it causes issues with TPU sharding
# flax.config.update('flax_always_shard_variable', False)

import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma import model as gemma_lib
from tunix.models.gemma import params as params_lib
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger

print("✓ All imports successful")
print(f"JAX version: {jax.__version__}")
print(f"Flax version: {flax.__version__}")
print(f"JAX devices: {jax.devices()}")

✓ All imports successful
JAX version: 0.8.0
Flax version: 0.12.0


E0000 00:00:1763157768.191835      12 common_lib.cc:648] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:238


JAX devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=4, process_index=0, coords=(0,2,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(1,2,0), core_on_chip=0), TpuDevice(id=6, process_index=0, coords=(0,3,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,3,0), core_on_chip=0)]


## Hyperparameters

In [7]:
# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

# ====== LoRA ======
RANK = 64
ALPHA = 64.0

# ====== Sharding ======
MESH = [(1, 4), ("fsdp", "tp")]

# ====== GRPO ======
# === Generation during GRPO training ===
MAX_PROMPT_LENGTH = 256
TOTAL_GENERATION_STEPS = 256
# Important to keep a high-ish temperature for varied, diverse responses during training.
TEMPERATURE = 0.9
TOP_P = 1.0
TOP_K = 50
# The number of times the policy generates multiple responses for a given prompt
# within a single training step.
NUM_GENERATIONS = 2

# === other GRPO configs ===
# The number of iterations per batch (𝜇 in GRPO algo 1).
NUM_ITERATIONS = 1
# The coefficient for the KL divergence penalty (𝛽) in the GRPO loss function.
BETA = 0.08
# Epsilon value for clipping (𝜀 in GRPO loss in paper).
EPSILON = 0.2

# ====== Training ======
TRAIN_MICRO_BATCH_SIZE = 1
# Reduce number of batches significantly to avoid OOM
NUM_BATCHES = 100
NUM_TEST_BATCHES = 50
EVAL_EVERY_N_STEPS = 10
NUM_EPOCHS = 1

# Number of training steps.
MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1
WARMUP_STEPS = 0.1 * MAX_STEPS
MAX_GRAD_NORM = 0.1

# Checkpoint saving
INTERMEDIATE_CKPT_DIR = "/tmp/content/intermediate_ckpt/"
CKPT_DIR = "/tmp/content/ckpts/"
SAVE_INTERVAL_STEPS = 50
MAX_TO_KEEP = 2

# ====== Inference ======
GENERATION_CONFIGS = {
    "greedy": {"temperature": 0.3, "top_k": 10, "top_p": 0.9},
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

print(f"Configuration set:")
print(f"  MAX_STEPS: {MAX_STEPS}")
print(f"  TRAIN_MICRO_BATCH_SIZE: {TRAIN_MICRO_BATCH_SIZE}")
print(f"  NUM_GENERATIONS: {NUM_GENERATIONS}")
print(f"  TOTAL_GENERATION_STEPS: {TOTAL_GENERATION_STEPS}")

Configuration set:
  MAX_STEPS: 100
  TRAIN_MICRO_BATCH_SIZE: 1
  NUM_GENERATIONS: 2
  TOTAL_GENERATION_STEPS: 256


## Memory usage

In [8]:
def show_hbm_usage():
  """Displays memory usage per device."""
  fmt_size = functools.partial(humanize.naturalsize, binary=True)

  for d in jax.local_devices():
    stats = d.memory_stats()
    used = stats["bytes_in_use"]
    limit = stats["bytes_limit"]
    print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")

## Data preprocessing

In [9]:
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"


SYSTEM_PROMPT = f"""You are given a problem. Think about the problem and \
provide your reasoning. Place it between {reasoning_start} and \
{reasoning_end}. Then, provide the final answer (i.e., just one numerical \
value) between {solution_start} and {solution_end}."""

TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

## The Devastator is here 

Original dataset was published by The Devastator (OpenAI's GSM8K dataset)

thedevastator/grade-school-math-8k-q-a

In [10]:
def extract_hash_answer(text: str) -> str | None:
  if "####" not in text:
    return None
  return text.split("####")[1].strip()


def _load_from_tfds(data_dir: str, split: str):
  import tensorflow_datasets.text.gsm8k
  return tfds.data_source(
      "gsm8k",
      split=split,
      data_dir=data_dir,
      builder_kwargs={"file_format": tfds.core.FileFormat.ARRAY_RECORD},
      download=True,
  )


def download_kaggle_dataset(target_dir="./data/gsm8k"):
  os.makedirs(target_dir, exist_ok=True)
  src = kagglehub.dataset_download("thedevastator/grade-school-math-8k-q-a")
  src = Path(src)
  dst = Path(target_dir)

  for csv_file in src.glob("*.csv"):  # match all CSV files
    shutil.copy2(csv_file, dst / csv_file.name)
    print(f"Copied {csv_file.name} → {dst/csv_file.name}")
  return target_dir


def get_dataset(data_dir, split="train", source="tfds") -> grain.MapDataset:
  # Download data
  if not os.path.exists(data_dir):
    os.makedirs(data_dir)

  if source == "tfds":
    import tensorflow_datasets.text.gsm8k
    data = tfds.data_source(
        "gsm8k",
        split=split,
        data_dir=data_dir,
        builder_kwargs={"file_format": tfds.core.FileFormat.ARRAY_RECORD},
        download=True,
    )

  elif source == "kaggle":
    kaggle_dir = download_kaggle_dataset(data_dir)
    file_name = "main_" + split + ".csv"
    csv_path = os.path.join(kaggle_dir, file_name)  # adjust filename if needed

    data = []
    with open(csv_path, newline="", encoding="utf-8") as csvfile:
      reader = csv.DictReader(csvfile)
      for row in reader:
        data.append({
            "question": row["question"],
            "answer": row["answer"],
        })

  else:
    raise ValueError(f"Unknown source: {source}")

  def _as_text(v):
    return v if isinstance(v, str) else v.decode("utf-8")

  dataset = (
      grain.MapDataset.source(data)
      .shuffle(seed=42)
      .map(
          lambda x: {
              # passed to model forward pass
              "prompts": TEMPLATE.format(
                  system_prompt=SYSTEM_PROMPT,
                  question=_as_text(x["question"]),
              ),
              # passed to reward functions
              "question": _as_text(x["question"]),
              # passed to reward functions
              "answer": extract_hash_answer(_as_text(x["answer"])),
          }
      )
  )
  return dataset

## Only Adding some Dataset we can go forward.

In [11]:
# source = input("Choose data source [tfds/kaggle]: ").strip().lower()
source = 'kaggle'

if source not in ("tfds", "kaggle"):
  print("Invalid choice. Defaulting to 'tfds'.")
  source = "tfds"

print(f"Using data source: {source}")

dataset = get_dataset(TRAIN_DATA_DIR, "train", source).batch(TRAIN_MICRO_BATCH_SIZE)[
    :NUM_BATCHES
]

if TRAIN_FRACTION == 1.0:
  train_dataset = dataset.repeat(NUM_EPOCHS)
  val_dataset = None
else:
  train_dataset = dataset[: int(len(dataset) * TRAIN_FRACTION)]
  train_dataset = train_dataset.repeat(NUM_EPOCHS)

  val_dataset = dataset[int(len(dataset) * TRAIN_FRACTION) :].repeat(NUM_EPOCHS)

test_dataset = get_dataset(TEST_DATA_DIR, "test", source).batch(TRAIN_MICRO_BATCH_SIZE)[
    :NUM_TEST_BATCHES
]

dataset_lengths = (
    len(train_dataset),
    len(val_dataset) if val_dataset is not None else 0,
    len(test_dataset),
)
print(f"dataset contains {dataset_lengths} of batches")

Using data source: kaggle
Copied main_test.csv → data/train/main_test.csv
Copied main_train.csv → data/train/main_train.csv
Copied socratic_train.csv → data/train/socratic_train.csv
Copied socratic_test.csv → data/train/socratic_test.csv
Copied main_test.csv → data/test/main_test.csv
Copied main_train.csv → data/test/main_train.csv
Copied socratic_train.csv → data/test/socratic_train.csv
Copied socratic_test.csv → data/test/socratic_test.csv
dataset contains (100, 0, 50) of batches


## One batch of the training dataset looks like.

In [12]:
for ele in train_dataset[:1]:
  pprint(ele)

{'answer': array(['3'], dtype='<U1'),
 'prompts': array(['<start_of_turn>user\nYou are given a problem. Think about the problem and provide your reasoning. Place it between <reasoning> and </reasoning>. Then, provide the final answer (i.e., just one numerical value) between <answer> and </answer>.\n\nMaria has 4 dimes, 4 quarters, and 7 nickels in her piggy bank. Her mom gives her 5 quarters. How much money, in dollars, does Maria have now?<end_of_turn>\n<start_of_turn>model'],
      dtype='<U417'),
 'question': array(['Maria has 4 dimes, 4 quarters, and 7 nickels in her piggy bank. Her mom gives her 5 quarters. How much money, in dollars, does Maria have now?'],
      dtype='<U142')}


In [13]:
from kaggle_secrets import UserSecretsClient

In [14]:
# Log in
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
  kagglehub.login()

## Gemma2": "google/gemma-2/flax

In [15]:
model_path = {
    "gemma2": "google/gemma-2/flax/",
}
model_family = "gemma2"
model_version = "gemma2-2b-it"
print(f"{model_path[model_family]}{model_version}")

kaggle_ckpt_path = kagglehub.model_download(
    f"{model_path[model_family]}{model_version}"
)

google/gemma-2/flax/gemma2-2b-it


## TPU is required here.

In [16]:
!rm /tmp/content/intermediate_ckpt/* -rf
!rm /tmp/content/ckpts/* -rf

if model_family == "gemma2":
    print("Loading Gemma2-2B-IT model parameters...")
    params = params_lib.load_and_format_params(
        os.path.join(kaggle_ckpt_path, "gemma2-2b-it")
    )
    print("✓ Parameters loaded")
    
    print("Creating Transformer model...")
    gemma = gemma_lib.Transformer.from_params(params, version="2-2b-it")
    print("✓ Transformer created")
    
    print("Saving intermediate checkpoint...")
    checkpointer = ocp.StandardCheckpointer()
    _, state = nnx.split(gemma)
    
    # Save synchronously to avoid async errors
    checkpointer.save(
        os.path.join(INTERMEDIATE_CKPT_DIR, "state"), 
        state,
        force=True  # Force synchronous save
    )
    
    # Wait for completion (this may show warnings but is safe)
    try:
        checkpointer.wait_until_finished()
        print("✓ Checkpoint saved")
    except Exception as e:
        print(f"⚠ Async warning (can be ignored): {e}")
        # Check if file was actually saved
        if os.path.exists(os.path.join(INTERMEDIATE_CKPT_DIR, "state")):
            print("✓ Checkpoint file verified to exist")
        else:
            raise RuntimeError("Checkpoint save failed!")
    
    # Clean up memory
    del params
    del gemma
    del state
    gc.collect()
    print("✓ Memory cleaned")

Loading Gemma2-2B-IT model parameters...
✓ Parameters loaded
Creating Transformer model...
✓ Transformer created
Saving intermediate checkpoint...
✓ Checkpoint saved
✓ Memory cleaned


##  LoRA Application

In [ ]:
def get_gemma_ref_model(ckpt_path, mesh): 
    model_config = gemma_lib.ModelConfig.gemma2_2b()
    abs_gemma: nnx.Module = nnx.eval_shape(
        lambda: gemma_lib.Transformer(model_config, rngs=nnx.Rngs(params=0))
    )
    abs_state = nnx.state(abs_gemma)
    abs_state = jax.tree.map(
        lambda a, s: jax.ShapeDtypeStruct(a.shape, jnp.bfloat16, sharding=s),
        abs_state,
        nnx.get_named_sharding(abs_state, mesh),
    )
    checkpointer = ocp.StandardCheckpointer()
    restored_params = checkpointer.restore(ckpt_path, target=abs_state)
    
    graph_def, _ = nnx.split(abs_gemma)
    gemma = nnx.merge(graph_def, restored_params)
    return gemma, model_config 


def get_lora_model(base_model, mesh):
    lora_provider = qwix.LoraProvider(
        module_path=(
            ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
            ".*attn_vec_einsum"
        ),
        rank=RANK,
        alpha=ALPHA,
    )
    
    model_input = base_model.get_model_input()
    lora_model = qwix.apply_lora_to_model(
        base_model, lora_provider, **model_input
    )
    
    with mesh:
        state = nnx.state(lora_model)
        pspecs = nnx.get_partition_spec(state)
        sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
        nnx.update(lora_model, sharded_state)
    
    return lora_model

In [ ]:
# Create mesh FIRST (before loading model)
print("Creating TPU mesh...")
mesh = jax.make_mesh(*MESH)
print(f"✓ Mesh created: {mesh}")

# Reference model
if model_family == "gemma2":
    print("Loading reference model...")
    ref_model, model_config = get_gemma_ref_model(
        ckpt_path=os.path.join(INTERMEDIATE_CKPT_DIR, "state"),
        mesh=mesh  # Pass mesh as argument
    )
    print("✓ Reference model loaded")

## LoRA policy

In [ ]:
# Policy model
lora_policy = get_lora_model(ref_model, mesh=mesh)
nnx.display(lora_policy)

## Tokenizer

In [ ]:
# Tokenizer
import sentencepiece as spm
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(os.path.join(kaggle_ckpt_path, "tokenizer.model"))
print("Loaded tokenizer using SentencePieceProcessor")

## Reward Function

In [ ]:
import re

# Define regex patterns
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)

match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})", flags=re.MULTILINE | re.DOTALL
)

# Reward function 1: Match format exactly (3 points)
def match_format_exactly(prompts, completions, **kwargs):
    return [
        0 if match_format.search(response) is None else 3.0
        for response in completions
    ]

# Reward function 2: Match format approximately (partial points)
def match_format_approximately(prompts, completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        score += 0.5 if completion.count(reasoning_start) == 1 else -0.5
        score += 0.5 if completion.count(reasoning_end) == 1 else -0.5
        score += 0.5 if completion.count(solution_start) == 1 else -0.5
        score += 0.5 if completion.count(solution_end) == 1 else -0.5
        scores.append(score)
    return scores

# Reward function 3: Check answer correctness
def check_answer(prompts, completions, answer, **kwargs):
    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None
        for r in completions
    ]
    
    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        if guess == true_answer:
            score += 3.0
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if ratio >= 0.9 and ratio <= 1.1:
                    score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2:
                    score += 0.25
                else:
                    score -= 1.0
            except:
                score -= 0.5
        scores.append(score)
    return scores

# Reward function 4: Check extracted numbers
def check_numbers(prompts, completions, answer, **kwargs):
    question = kwargs["question"]
    
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None
        for r in completions
    ]
    
    scores = []
    if len(question) > 0 and len(answer) > 0 and len(completions) > 0:
        print("START ============================")
        print(f"Question: {question[0]}")
        print(f"Answer: {answer[0]}")
        print(f"Response: {completions[0][:100]}...")
        print(f"Extracted: {extracted_responses[0]}")
        print("END ==============================")
    
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        try:
            true_answer_float = float(true_answer.strip())
            guess_float = float(guess.strip())
            scores.append(1.5 if guess_float == true_answer_float else 0.0)
        except:
            scores.append(0)
    return scores

print("✓ Reward functions defined (4 functions)")

## Optimizer

In [ ]:
# Optimizer with warmup cosine decay and gradient clipping
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)
if MAX_GRAD_NORM is not None:
    optimizer = optax.chain(
        optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
        optimizer,
    )
print("✓ Optimizer created")

# Checkpoint saving options
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS, 
    max_to_keep=MAX_TO_KEEP
)
print("✓ Checkpointing options created")

# Metrics logging options - DISABLED to avoid wandb issues
metrics_logging_options = None  # Disable WandB logging
print("✓ Metrics logging options created (WandB disabled)")

# Cluster configuration
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine='vanilla',
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=MAX_STEPS,
        mini_batch_size=TRAIN_MICRO_BATCH_SIZE,
        train_micro_batch_size=TRAIN_MICRO_BATCH_SIZE,
        metrics_logging_options=metrics_logging_options,  # None = no WandB
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
    ),
)
print("✓ ClusterConfig created")

# GRPO configuration
grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
)
print("✓ GRPOConfig created")

# RL cluster
rl_cluster = rl_cluster_lib.RLCluster(
    actor=lora_policy,
    reference=ref_model,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)
print("✓ RLCluster created")

# GRPO Trainer
grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=[
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    grpo_config=grpo_config,
)

print("\n✅ GRPO trainer created successfully!")

## Training

In [ ]:
print("Starting GRPO training...")
print(f"Max steps: {MAX_STEPS}")
print(f"Save interval: {SAVE_INTERVAL_STEPS} steps")
print(f"Eval every: {EVAL_EVERY_N_STEPS} steps")
print(f"Dataset batches: {len(train_dataset)}\n")

# Start training with mesh context
with mesh:
    grpo_trainer.train(train_dataset)

print("\n✅ Training completed!")

## Checkpoint check

In [ ]:
print("Loading trained checkpoint...")

# Auto-detect latest checkpoint
ckpt_base = os.path.join(CKPT_DIR, "actor")
saved_steps = sorted([int(d) for d in os.listdir(ckpt_base) if d.isdigit()])
latest_step = saved_steps[-1]

print(f"Available checkpoints: {saved_steps}")
print(f"Loading from step: {latest_step}")

trained_ckpt_path = os.path.join(ckpt_base, str(latest_step), "model_params")

abs_params = jax.tree.map(
    lambda x: jax.ShapeDtypeStruct(x.shape, x.dtype),
    nnx.state(lora_policy, nnx.LoRAParam),
)
checkpointer = ocp.StandardCheckpointer()
trained_lora_params = checkpointer.restore(trained_ckpt_path, target=abs_params)

nnx.update(
    lora_policy,
    jax.tree.map(lambda a, b: b, nnx.state(lora_policy, nnx.LoRAParam), trained_lora_params),
)

print(f"✓ Trained checkpoint loaded from step {latest_step}")

checkpoint_dir = os.path.join(ckpt_base, str(latest_step))
print(f"\nCheckpoint contents:")
for item in os.listdir(checkpoint_dir):
    item_path = os.path.join(checkpoint_dir, item)
    if os.path.isfile(item_path):
        size = os.path.getsize(item_path) / (1024**2)
        print(f"  {item}: {size:.2f} MB")
    else:
        print(f"  {item}/ (directory)")

## Mock Evaluation

In [ ]:
def generate(question, sampler, temperature=0.7, top_k=50, top_p=0.95, seed=None):
    """Given prompt, generates text."""
    if isinstance(question, str):
        input_batch = [
            TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=question),
        ]
    else:
        input_batch = [
            TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=q)
            for q in question
        ]
    
    out_data = sampler(
        input_strings=input_batch,
        max_generation_steps=768,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        echo=False,
        seed=seed,
    )
    
    output = out_data.text
    return output[0] if isinstance(question, str) else output


def evaluate(dataset, sampler, temperature=0.7, top_k=50, top_p=0.95, num_passes=1):
    """Computes accuracy and format matching percentage."""
    corr = 0
    partially_corr = 0
    corr_format = 0
    total = 0
    
    for batch in tqdm(dataset):
        answers = batch["answer"]
        questions = batch["question"]
        
        for p in range(num_passes):
            responses = generate(questions, sampler, temperature, top_k, top_p, seed=p)
            
            for response, answer in zip(responses, answers):
                extracted = (
                    guess.group(1)
                    if (guess := match_numbers.search(response)) is not None
                    else "-1000000"
                )
                try:
                    if float(extracted.strip()) == float(answer.strip()):
                        corr += 1
                    ratio = float(extracted.strip()) / float(answer.strip())
                    if 0.9 <= ratio <= 1.1:
                        partially_corr += 1
                except:
                    pass
                
                if match_format.search(response) is not None:
                    corr_format += 1
                
                total += 1
                if total % 10 == 0:
                    print(f"Progress: {corr}/{total} correct ({corr/total*100:.1f}%)")
    
    return (
        corr, total, 
        corr / total * 100, 
        partially_corr / total * 100, 
        corr_format / total * 100
    )

print("✓ Evaluation functions defined")

In [ ]:
# Create sampler with trained model
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=1536,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

# Evaluate on test dataset
print("\nEvaluating trained model on test set...")
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset, sampler, **GENERATION_CONFIGS["greedy"]
)
print(f"\n{'='*50}")
print(f"FINAL RESULTS:")
print(f"Correct: {corr}/{total}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Partial Accuracy: {partial_accuracy:.2f}%")
print(f"Format Accuracy: {format_accuracy:.2f}%")
print(f"{'='*50}")

In [ ]:
# Create sampler with trained model
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=1536,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

# Evaluate on test dataset
print("\nEvaluating trained model on test set...")
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset, sampler, **GENERATION_CONFIGS["standard"]
)
print(f"\n{'='*50}")
print(f"FINAL RESULTS:")
print(f"Correct: {corr}/{total}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Partial Accuracy: {partial_accuracy:.2f}%")
print(f"Format Accuracy: {format_accuracy:.2f}%")
print(f"{'='*50}")

In [ ]:
# Create sampler with trained model
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=1536,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

# Evaluate on test dataset
print("\nEvaluating trained model on test set...")
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset, sampler, **GENERATION_CONFIGS["liberal"]
)
print(f"\n{'='*50}")
print(f"FINAL RESULTS:")
print(f"Correct: {corr}/{total}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Partial Accuracy: {partial_accuracy:.2f}%")
print(f"Format Accuracy: {format_accuracy:.2f}%")
print(f"{'='*50}")

In [ ]:
# Generate sample outputs for demonstration
sample_questions = [
    "What is 15 + 27?",
    "If a train travels 60 miles per hour for 3 hours, how far does it travel?",
    "What is 5 factorial (5!)?",
    "A rectangle has length 8 cm and width 5 cm. What is its area?",
    "If John has 3 apples and Mary gives him 4 more, how many apples does John have?",
]

print("="*80)
print("SAMPLE MODEL OUTPUTS - Demonstrating Reasoning Capability")
print("="*80)

for i, q in enumerate(sample_questions, 1):
    prompt = TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=q)
    out = sampler(
        input_strings=[prompt],
        max_generation_steps=512,
        **GENERATION_CONFIGS["standard"]
    )
    
    response = out.text[0]
    
    print(f"\n{'='*80}")
    print(f"Example {i}:")
    print(f"Question: {q}")
    print(f"\nModel Response:")
    print(response)
    
    # Check format
    has_reasoning = '<reasoning>' in response and '</reasoning>' in response
    has_answer = '<answer>' in response and '</answer>' in response
    print(f"\n✓ Format check: Reasoning={'✓' if has_reasoning else '✗'}, Answer={'✓' if has_answer else '✗'}")
    
print(f"\n{'='*80}")
print("END OF SAMPLE OUTPUTS")
print("="*80)

In [17]:
# ============================================================================
# CELL: Test Model on Diverse Domains (Competition Evaluation Preview)
# ============================================================================
# Competition evaluates on diverse domains, not just math:
# - Creative writing, Creative ideation, Summarization
# - Math, Coding, Basic science, Other
#
# This cell tests whether GSM8K-only training is sufficient
# ============================================================================

# Diverse evaluation prompts
DIVERSE_TEST_PROMPTS = [
    # Math
    {
        "domain": "Math",
        "question": "If a rectangular garden is 12 meters long and 8 meters wide, what is its perimeter?",
    },
    # Creative Writing
    {
        "domain": "Creative Writing",
        "question": "Write a short opening paragraph for a story about a detective who discovers that shadows are actually alive.",
    },
    # Creative Ideation
    {
        "domain": "Creative Ideation",
        "question": "Brainstorm 3 innovative features for a smart water bottle designed for marathon runners.",
    },
    # Summarization
    {
        "domain": "Summarization",
        "question": "Summarize this text in 2-3 sentences: 'Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water. Photosynthesis in plants generally involves the green pigment chlorophyll and generates oxygen as a byproduct. This process is fundamental to life on Earth as it provides the primary source of organic compounds and oxygen in the atmosphere.'",
    },
    # Coding
    {
        "domain": "Coding",
        "question": "Write a Python function that checks if a given string is a palindrome (reads the same forwards and backwards). Explain your approach.",
    },
    # Basic Science
    {
        "domain": "Basic Science",
        "question": "Explain why the sky appears blue during the day but turns red and orange during sunset.",
    },
    # Logic/Reasoning
    {
        "domain": "Logic",
        "question": "If all roses are flowers, and some flowers fade quickly, can we conclude that some roses fade quickly?",
    },
]

print("="*80)
print("DIVERSE DOMAIN EVALUATION - Preview of Competition Evaluation")
print("="*80)
print("Testing model's ability to generalize beyond math problems")
print(f"Total test cases: {len(DIVERSE_TEST_PROMPTS)} domains\n")

results = []
format_pass_count = 0

for i, test_case in enumerate(DIVERSE_TEST_PROMPTS, 1):
    domain = test_case["domain"]
    question = test_case["question"]

    # Generate response using sampler
    prompt = TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=question)
    out = sampler(
        input_strings=[prompt],
        max_generation_steps=768,
        echo=False,
        seed=42,
        **GENERATION_CONFIGS["standard"]  # temp=0.7
    )

    response = out.text[0]

    # Check format compliance
    has_reasoning = '<reasoning>' in response and '</reasoning>' in response
    has_answer = '<answer>' in response and '</answer>' in response
    format_ok = has_reasoning and has_answer

    if format_ok:
        format_pass_count += 1

    # Print output
    print(f"{'='*80}")
    print(f"Test {i}/7 - Domain: {domain}")
    print(f"{'='*80}")
    print(f"Question: {question[:100]}{'...' if len(question) > 100 else ''}")
    print(f"\nModel Response:")
    print(response)
    print(f"\n{'—'*40}")
    print(f"Format: {'✓ PASS' if format_ok else '✗ FAIL'} (Reasoning: {'✓' if has_reasoning else '✗'}, Answer: {'✓' if has_answer else '✗'})")
    print()

# Summary
format_pass_rate = format_pass_count / len(DIVERSE_TEST_PROMPTS) * 100

print(f"{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"Format compliance: {format_pass_count}/{len(DIVERSE_TEST_PROMPTS)} ({format_pass_rate:.1f}%)")

if format_pass_rate >= 85:
    print("\n✓ EXCELLENT: Model shows strong format compliance across domains")
    print("  → GSM8K-only training may be sufficient for format learning")
    print("  → Focus on improving reasoning quality")
elif format_pass_rate >= 60:
    print("\n⚠ MODERATE: Model shows partial format compliance")
    print("  → Consider adding diverse training examples")
    print("  → Or increase GRPO training steps on GSM8K")
else:
    print("\n✗ WEAK: Model struggles with format in diverse domains")
    print("  → Likely needs multi-domain training data")
    print("  → GSM8K-only training insufficient")

print(f"{'='*80}")


DIVERSE DOMAIN EVALUATION - Preview of Competition Evaluation
Testing model's ability to generalize beyond math problems
Total test cases: 7 domains



NameError: name 'sampler' is not defined

## Kaggle model ID